In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn import tree
import matplotlib.pyplot as plt
import pydotplus
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential

In [56]:
df = pd.read_excel('../data_final.xlsx')
df=df.iloc[:,1:]
df_org = df.copy()
df.head(3)

,Duration,Resource Cost,Fixed Cost,Cost/Hour,Variable Cost,Total Cost,Actual Cost
0,3.0,2400.0,0.0,0.0,0.0,2400.0,2400.0
1,1.0,800.0,0.0,0.0,0.0,800.0,800.0
2,1.0,440.0,0.0,0.0,0.0,440.0,440.0


In [50]:
#Standard
for column in df.columns:
    df[column] = MinMaxScaler().fit_transform(np.array(df[column]).reshape(-1,1))
df.head(10)

,Duration,Resource Cost,Fixed Cost,Cost/Hour,Variable Cost,Total Cost,Actual Cost
0,0.031637,0.038095,0.000000e+00,0.0,0.0,2.149177e-03,2.149177e-03
1,0.009629,0.012698,0.000000e+00,0.0,0.0,7.163863e-04,7.163863e-04
2,0.009629,0.006984,0.000000e+00,0.0,0.0,3.940084e-04,3.940084e-04
3,0.000000,0.000000,9.328654e-07,0.0,0.0,8.865392e-07,8.865392e-07
4,0.004127,0.005841,8.878677e-03,0.0,0.0,8.852523e-03,8.852523e-03
5,0.009629,0.017524,7.204146e-03,0.0,0.0,7.904159e-03,7.904159e-03
6,0.196699,0.315429,8.191025e-04,0.0,0.0,1.858154e-02,1.858154e-02
7,0.086657,0.168229,3.862063e-04,0.0,0.0,9.861530e-03,9.861530e-03
8,0.053645,0.105143,3.358315e-05,0.0,0.0,5.963982e-03,5.963982e-03
9,0.053645,0.090794,7.749574e-03,0.0,0.0,1.256134e-02,1.256134e-02


In [57]:
# setting x and y
#X = df.drop('Actual Cost',axis=1)
X = df[['Duration','Total Cost']]
y = df['Actual Cost']
# preparing test and train
X_train,X_test,y_train,y_test = train_test_split(
    X,y,test_size=0.3
)
X.shape

(3074, 2)

In [58]:
# create model
model = Sequential()
model.add(Dense(6, input_dim=2,  activation='linear'))
model.add(Dense(6,activation='linear'))
model.add(Dense(1,activation='linear'))
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X,y,epochs=5)
y_pred = model.predict(X)
print("MAE ANN ON TEST:")
print(MAE(y,y_pred))

Epoch 1/5
97/97 [==============================] - 0s 979us/step - loss: 1812578048.0000
Epoch 2/5
97/97 [==============================] - 0s 1ms/step - loss: 690667200.0000
Epoch 3/5
97/97 [==============================] - 0s 999us/step - loss: 677332096.0000
Epoch 4/5
97/97 [==============================] - 0s 968us/step - loss: 674491328.0000
Epoch 5/5
97/97 [==============================] - 0s 1ms/step - loss: 680747968.0000
MAE ANN ON TEST:
5099.850458903961


In [59]:
xg_reg = xgb.XGBRegressor()
xg_reg.fit(X,y)
y_pred = xg_reg.predict(X)
print("MAE XGB ON TEST:")
print(MAE(y,y_pred))
model = RandomForestRegressor()
cross = cross_val_score(model,X,y,cv=3,scoring='neg_mean_absolute_error')
print("MAE RF CROSS-VALIDATION:")
print(-np.average(cross))
model.fit(X,y)
y_pred = model.predict(X)
print("MAE RF ON TEST:")
print(MAE(y,y_pred))
MAE_MSP = MAE(df['Actual Cost'],df['Total Cost'])
print("MAE MSP:")
print(MAE_MSP)

MAE XGB ON TEST:
2023.9696802180272
MAE RF CROSS-VALIDATION:
7704.293411769661
MAE RF ON TEST:
2828.001580424186
MAE MSP:
4795.18957719731


RandomForest HyperParameter Tuning

In [6]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 30, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)
rf_random.best_params_

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.2min finished


{'n_estimators': 1800,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 90,
 'bootstrap': True}

## Trying to find the best hyper paramteters

In [ ]:
splitter = ['best','random']
max_depth=[None,10,20,50,100,150,200,500]
min_samples_split=[1,2,5,10,20]
min_samples_leaf = [1,2,5,10,20]
max_features = ['auto','sqrt','log2']
max_leaf_nodes = [None,100,500,1000]
ccp_alpha = [0,1,5,10,20]
min_impurity_decrease = [0,10,20,100,200,500]
random_grid = {
    'splitter':splitter,
    'max_depth':max_depth,
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf,
    'max_features':max_features,
    'max_leaf_nodes':max_leaf_nodes,
    'ccp_alpha':ccp_alpha,
    'min_impurity_decrease':min_impurity_decrease
}
tree_regressor = DecisionTreeRegressor()
random_search = RandomizedSearchCV(
    estimator=tree_regressor,
    param_distributions=random_grid,
    n_iter=30,
    cv=3,
    random_state=42,
    n_jobs=-1,
    verbose=2,
    scoring='neg_mean_absolute_error'
)
random_search.fit(X_train,y_train)
print("done")

In [55]:
print(random_search.best_score_)

-11705.91433417153


In [47]:
cross = cross_val_score(tree_regressor,X_train,y_train,cv=5,scoring='neg_mean_absolute_error')
print(-np.average(cross))

9736.453763354217


In [141]:
# Defining Model
tree_regressor = DecisionTreeRegressor(random_state=42)
# fitting
tree_regressor.fit(X,y)
# R2 scores
train_r2 = tree_regressor.score(X_train,y_train)
test_r2 = tree_regressor.score(X_test,y_test)
print("Train R2: "+ str(train_r2))
print("Test R2: "+ str(test_r2))
y_pred = tree_regressor.predict(X_test)
error = MSE(y_test,y_pred)
print('MSE:')
print(error)
error_MAE = MAE(y_pred,y_test)
print("MAE:")
print(error_MAE)

Train R2: 0.9837175411327904
Test R2: 0.9709122363430546
MSE:
2725765.5829482283
MAE:
391.2401493599831


In [100]:
MAE_MSP = MAE(df['Actual Cost'],df['Total Cost'])
print("MAE MSP:")
print(MAE_MSP)
MSE_MSP = MSE(df['Actual Cost'],df['Total Cost'])
print("MSE MSP:")
print(MSE_MSP)

MAE MSP:
1390.034525965575
MSE MSP:
17489863.58723825


In [6]:
# number of leaves
print(tree_regressor.tree_.n_leaves)

1987


In [7]:
# Saving Tree Graph
dot_data = tree.export_graphviz(tree_regressor, out_file=None, 
                                feature_names=['Duration','Total Cost'],class_names=['Acutal Cost'])
graph = pydotplus.graph_from_dot_data(dot_data)  
graph.write_svg("tree.svg")


True

In [8]:
# Saving model
pkl_filename = "tree.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(tree_regressor, file)

In [9]:
pkl_filename = "tree.pkl"
with open(pkl_filename,'rb') as file:
    model = pickle.load(file)
print("MODEL IMPORTED!")

MODEL IMPORTED!


In [10]:
model.predict([[100,100000000000000000000000000]])


array([42642.])